In [1]:
import requests
import re
import json
import os
import numpy as np
import pandas as pd
import requests
from io import StringIO

# Gathering data

In [2]:
#WeRateDogs dataset
df_WeRateDogs = pd.read_csv('twitter-archive-enhanced.csv')

#imagePrediction dataset
r = requests.get('https://d17h27t6h515a5.cloudfront.net/topher/2017/August/599fd2ad_image-predictions/image-predictions.tsv')
df_imagePrediction = pd.read_csv(StringIO(r.text),sep='\t')

#tweet-json dataset(json format, needs further cleaning)
#Tried to apply for API from twitter, but I don't know why even with the reason provided, they still doesn't pass my 
#application
list_tweepy = []
with open('tweet-json.txt', encoding='utf8') as f:
    for each in f:
        data = json.loads(each)
        list_tweepy.append(data)

# Assessing data

In [3]:
#WeRateDogs
df_WeRateDogs.sample()

,tweet_id,in_reply_to_status_id,in_reply_to_user_id,timestamp,source,text,retweeted_status_id,retweeted_status_user_id,retweeted_status_timestamp,expanded_urls,rating_numerator,rating_denominator,name,doggo,floofer,pupper,puppo
292,838083903487373313,NaN,NaN,2017-03-04 17:49:08 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Daisy. She's puppears to be rare as al...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/838083903...,13,10,Daisy,None,None,None,None


In [4]:
df_WeRateDogs.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2356 entries, 0 to 2355
Data columns (total 17 columns):
tweet_id                      2356 non-null int64
in_reply_to_status_id         78 non-null float64
in_reply_to_user_id           78 non-null float64
timestamp                     2356 non-null object
source                        2356 non-null object
text                          2356 non-null object
retweeted_status_id           181 non-null float64
retweeted_status_user_id      181 non-null float64
retweeted_status_timestamp    181 non-null object
expanded_urls                 2297 non-null object
rating_numerator              2356 non-null int64
rating_denominator            2356 non-null int64
name                          2356 non-null object
doggo                         2356 non-null object
floofer                       2356 non-null object
pupper                        2356 non-null object
puppo                         2356 non-null object
dtypes: float64(4), int64(3), ob

In [5]:
df_WeRateDogs[['rating_numerator','rating_denominator']].describe()

,rating_numerator,rating_denominator
count,2356.000000,2356.000000
mean,13.126486,10.455433
std,45.876648,6.745237
min,0.000000,0.000000
25%,10.000000,10.000000
50%,11.000000,10.000000
75%,12.000000,10.000000
max,1776.000000,170.000000


In [6]:
len(df_WeRateDogs[df_WeRateDogs['retweeted_status_id'].isnull()==False])

181

In [7]:
print(df_WeRateDogs[['tweet_id']].shape[0])
print(df_WeRateDogs[['tweet_id']].drop_duplicates().shape[0])

print()

print(df_WeRateDogs['expanded_urls'].shape[0])
print(df_WeRateDogs['expanded_urls'].drop_duplicates().shape[0])

2356
2356

2356
2219


In [8]:
#imagePrediction
df_imagePrediction.sample()

,tweet_id,jpg_url,img_num,p1,p1_conf,p1_dog,p2,p2_conf,p2_dog,p3,p3_conf,p3_dog
118,668154635664932864,https://pbs.twimg.com/media/CUXDGR2WcAAUQKz.jpg,1,Arctic_fox,0.473584,False,wallaby,0.261411,False,white_wolf,0.080948,False


In [9]:
df_imagePrediction.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2075 entries, 0 to 2074
Data columns (total 12 columns):
tweet_id    2075 non-null int64
jpg_url     2075 non-null object
img_num     2075 non-null int64
p1          2075 non-null object
p1_conf     2075 non-null float64
p1_dog      2075 non-null bool
p2          2075 non-null object
p2_conf     2075 non-null float64
p2_dog      2075 non-null bool
p3          2075 non-null object
p3_conf     2075 non-null float64
p3_dog      2075 non-null bool
dtypes: bool(3), float64(3), int64(2), object(4)
memory usage: 152.1+ KB


In [10]:
df_imagePrediction[['p1_conf','p2_conf','p3_conf']].describe()

,p1_conf,p2_conf,p3_conf
count,2075.000000,2.075000e+03,2.075000e+03
mean,0.594548,1.345886e-01,6.032417e-02
std,0.271174,1.006657e-01,5.090593e-02
min,0.044333,1.011300e-08,1.740170e-10
25%,0.364412,5.388625e-02,1.622240e-02
50%,0.588230,1.181810e-01,4.944380e-02
75%,0.843855,1.955655e-01,9.180755e-02
max,1.000000,4.880140e-01,2.734190e-01


In [11]:
print(df_imagePrediction['tweet_id'].shape[0])
print(df_imagePrediction['tweet_id'].drop_duplicates().shape[0])

print("")

print(df_imagePrediction['jpg_url'].shape[0])
print(df_imagePrediction['jpg_url'].drop_duplicates().shape[0])

2075
2075

2075
2009


In [12]:
#tweet_json(needs to convert to dataframe)
tweet_id = []
retweet_count = []
favourite_count = []

for each in range(len(list_tweepy)):
    tweet_id.append(list_tweepy[each]['id_str'])
    retweet_count.append(list_tweepy[each]['retweet_count'])
    favourite_count.append(list_tweepy[each]['favorite_count'])

df_tweepy={'tweet_id':tweet_id,
        'retweet_count':retweet_count,
        'favourite_count':favourite_count}

df_tweepy = pd.DataFrame.from_dict(df_tweepy)

In [13]:
df_tweepy.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2354 entries, 0 to 2353
Data columns (total 3 columns):
tweet_id           2354 non-null object
retweet_count      2354 non-null int64
favourite_count    2354 non-null int64
dtypes: int64(2), object(1)
memory usage: 55.2+ KB


In [14]:
df_tweepy.head()

,tweet_id,retweet_count,favourite_count
0,892420643555336193,8853,39467
1,892177421306343426,6514,33819
2,891815181378084864,4328,25461
3,891689557279858688,8964,42908
4,891327558926688256,9774,41048


In [15]:
print(df_tweepy['tweet_id'].shape[0])
print(df_tweepy['tweet_id'].drop_duplicates().shape[0])

2354
2354


# Cleaning Data

In [16]:
#1: Clean datatypes

#1.1:Change datatype of df_WeRateDogs
convert_dict={'tweet_id':'object',
              'in_reply_to_status_id':'object',
              'in_reply_to_user_id':'object',
              'rating_numerator':'float64',
              'rating_denominator':'float64'}
df_WeRateDogs = df_WeRateDogs.astype(convert_dict)
df_WeRateDogs['timestamp'] = pd.to_datetime(df_WeRateDogs['timestamp'])

#1.2:Change datatpe of df_imagePrediction
convert_dict = {'tweet_id':'object'}
df_imagePrediction = df_imagePrediction.astype(convert_dict)

In [17]:
#2: Remove duplicate rows
df_imagePrediction.drop_duplicates(subset=['jpg_url'], keep='last',inplace=True)
df_WeRateDogs.drop_duplicates(subset=['expanded_urls'], keep='last',inplace=True)

In [18]:
df_WeRateDogs.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2219 entries, 0 to 2355
Data columns (total 17 columns):
tweet_id                      2219 non-null object
in_reply_to_status_id         24 non-null object
in_reply_to_user_id           24 non-null object
timestamp                     2219 non-null datetime64[ns]
source                        2219 non-null object
text                          2219 non-null object
retweeted_status_id           102 non-null float64
retweeted_status_user_id      102 non-null float64
retweeted_status_timestamp    102 non-null object
expanded_urls                 2218 non-null object
rating_numerator              2219 non-null float64
rating_denominator            2219 non-null float64
name                          2219 non-null object
doggo                         2219 non-null object
floofer                       2219 non-null object
pupper                        2219 non-null object
puppo                         2219 non-null object
dtypes: datetime64[ns

In [19]:
df_imagePrediction.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2009 entries, 0 to 2074
Data columns (total 12 columns):
tweet_id    2009 non-null object
jpg_url     2009 non-null object
img_num     2009 non-null int64
p1          2009 non-null object
p1_conf     2009 non-null float64
p1_dog      2009 non-null bool
p2          2009 non-null object
p2_conf     2009 non-null float64
p2_dog      2009 non-null bool
p3          2009 non-null object
p3_conf     2009 non-null float64
p3_dog      2009 non-null bool
dtypes: bool(3), float64(3), int64(1), object(5)
memory usage: 162.8+ KB


In [20]:
print(df_imagePrediction['jpg_url'].shape[0])
print(df_imagePrediction['jpg_url'].drop_duplicates().shape[0])

2009
2009


In [21]:
print(df_WeRateDogs['expanded_urls'].shape[0])
print(df_WeRateDogs['expanded_urls'].drop_duplicates().shape[0])

2219
2219


## WeRateDogs

In [22]:
#1: Filter all non-null value to make sure we only have original tweet
# -use dataframe filter to remove the rows does not meet the requirement
df_WeRateDogs = df_WeRateDogs[df_WeRateDogs['retweeted_status_id'].isnull() == True]
df_WeRateDogs.drop(['retweeted_status_id','retweeted_status_user_id','retweeted_status_timestamp'], axis=1,inplace=True)

#test
df_WeRateDogs.head()

,tweet_id,in_reply_to_status_id,in_reply_to_user_id,timestamp,source,text,expanded_urls,rating_numerator,rating_denominator,name,doggo,floofer,pupper,puppo
0,892420643555336193,NaN,NaN,2017-08-01 16:23:56,"<a href=""http://twitter.com/download/iphone"" r...",This is Phineas. He's a mystical boy. Only eve...,https://twitter.com/dog_rates/status/892420643...,13.0,10.0,Phineas,None,None,None,None
1,892177421306343426,NaN,NaN,2017-08-01 00:17:27,"<a href=""http://twitter.com/download/iphone"" r...",This is Tilly. She's just checking pup on you....,https://twitter.com/dog_rates/status/892177421...,13.0,10.0,Tilly,None,None,None,None
2,891815181378084864,NaN,NaN,2017-07-31 00:18:03,"<a href=""http://twitter.com/download/iphone"" r...",This is Archie. He is a rare Norwegian Pouncin...,https://twitter.com/dog_rates/status/891815181...,12.0,10.0,Archie,None,None,None,None
3,891689557279858688,NaN,NaN,2017-07-30 15:58:51,"<a href=""http://twitter.com/download/iphone"" r...",This is Darla. She commenced a snooze mid meal...,https://twitter.com/dog_rates/status/891689557...,13.0,10.0,Darla,None,None,None,None
4,891327558926688256,NaN,NaN,2017-07-29 16:00:24,"<a href=""http://twitter.com/download/iphone"" r...",This is Franklin. He would like you to stop ca...,https://twitter.com/dog_rates/status/891327558...,12.0,10.0,Franklin,None,None,None,None


In [23]:
#2: clean numerator and denominator number

rating_numerator = []
rating_denominator = []
for each in df_WeRateDogs['text']:
    rating_numerator.append(each[each.find('/')-2:each.find('/')+3][:2])
    rating_denominator.append(each[each.find('/')-2:each.find('/')+3][3:])
    
count = 0
for each in rating_numerator:
    try:
        rating_numerator[count] = int(each)
    except:
        try:
            rating_numerator[count] = int(each[1])
        except:
            rating_numerator[count]='critical'
    count+=1
    
count = 0
for each in rating_denominator:
    try:
        rating_denominator[count] = int(each)
    except:
        rating_denominator[count] = 'critical'
    count+=1

df_WeRateDogs.drop(['rating_numerator','rating_denominator'], axis=1, inplace=True)
df_WeRateDogs['rating_numerator'] = rating_numerator
df_WeRateDogs['rating_denominator'] = rating_denominator

df_WeRateDogs[df_WeRateDogs['rating_numerator'] == 'critical']

,tweet_id,in_reply_to_status_id,in_reply_to_user_id,timestamp,source,text,expanded_urls,name,doggo,floofer,pupper,puppo,rating_numerator,rating_denominator
1249,711306686208872448,NaN,NaN,2016-03-19 21:41:44,"<a href=""http://twitter.com/download/iphone"" r...",What hooligan sent in pictures w/out a dog in ...,https://twitter.com/dog_rates/status/711306686...,None,None,None,None,None,critical,critical
1815,676613908052996102,NaN,NaN,2015-12-15 04:05:01,"<a href=""http://twitter.com/download/iphone"" r...",This is the saddest/sweetest/best picture I've...,https://twitter.com/dog_rates/status/676613908...,the,None,None,None,None,critical,critical


In [24]:
# -Due to some of numerator/denominator number are not accuracy, I found that we can use find function, 
# and extract the left side of foreslash as numerator, and use right side of foreslash as denominator. 
# It will solve most of the problems, however, there are still few rows with the problem cannot be solved, 
# after scanning the dataset, I found there are only 2 rows does not fit the solution. 
# Hence, I extract the twitter text to manually fill the numerator and denominator value.

for each in df_WeRateDogs[df_WeRateDogs['rating_denominator'] == 'critical']['text']:
    print(each)

What hooligan sent in pictures w/out a dog in them? Churlish af. 3/10 just bc that's a neat fluffy bean bag chair https://t.co/wcwoGOkZvz
This is the saddest/sweetest/best picture I've been sent. 12/10 😢🐶 https://t.co/vQ2Lw1BLBF


In [25]:
df_WeRateDogs.at[1249,'rating_denominator'] = 10
df_WeRateDogs.at[1815,'rating_denominator'] = 10
df_WeRateDogs.at[1249,'rating_numerator'] = 3
df_WeRateDogs.at[1815,'rating_numerator'] = 12

In [26]:
#test
df_WeRateDogs[df_WeRateDogs['rating_numerator'] == 'critical']

,tweet_id,in_reply_to_status_id,in_reply_to_user_id,timestamp,source,text,expanded_urls,name,doggo,floofer,pupper,puppo,rating_numerator,rating_denominator


In [27]:
#3: We found that there is 0 in rating denominator, which cause divide 0 error.
#-use dataframe filter to remove the rows that demoninator is 0.
df_WeRateDogs = df_WeRateDogs[df_WeRateDogs['rating_denominator'] != 0]
#test
list(set(df_WeRateDogs['rating_denominator']))

[2, 70, 7, 40, 10, 11, 12, 13, 15, 80, 17, 50, 20, 90]

In [28]:
# 4: Remove the columns that does not use in the case.
# -create a sub-dataset as the final version of dataset.
df_WeRateDogs = df_WeRateDogs[['tweet_id','timestamp','text','name',
                               'doggo','floofer','pupper','puppo',
                               'rating_numerator','rating_denominator']]
#test
df_WeRateDogs.head()

,tweet_id,timestamp,text,name,doggo,floofer,pupper,puppo,rating_numerator,rating_denominator
0,892420643555336193,2017-08-01 16:23:56,This is Phineas. He's a mystical boy. Only eve...,Phineas,None,None,None,None,13,10
1,892177421306343426,2017-08-01 00:17:27,This is Tilly. She's just checking pup on you....,Tilly,None,None,None,None,13,10
2,891815181378084864,2017-07-31 00:18:03,This is Archie. He is a rare Norwegian Pouncin...,Archie,None,None,None,None,12,10
3,891689557279858688,2017-07-30 15:58:51,This is Darla. She commenced a snooze mid meal...,Darla,None,None,None,None,13,10
4,891327558926688256,2017-07-29 16:00:24,This is Franklin. He would like you to stop ca...,Franklin,None,None,None,None,12,10


## imagePrediction 

In [29]:
# 5: some confidence number greater than 0
df_imagePrediction[['p1_conf','p2_conf','p3_conf']].describe()

,p1_conf,p2_conf,p3_conf
count,2009.000000,2.009000e+03,2.009000e+03
mean,0.594467,1.340727e-01,6.017061e-02
std,0.272236,1.006394e-01,5.085369e-02
min,0.044333,1.011300e-08,1.740170e-10
25%,0.362925,5.353130e-02,1.610520e-02
50%,0.588230,1.170030e-01,4.947150e-02
75%,0.847292,1.948970e-01,9.157150e-02
max,1.000000,4.880140e-01,2.734190e-01


In [30]:
df_imagePrediction['p2_conf'] = df_imagePrediction['p2_conf']/10
df_imagePrediction['p3_conf'] = df_imagePrediction['p3_conf']/10
#test
print(df_imagePrediction[df_imagePrediction['p2_conf'] > 1])
print(df_imagePrediction[df_imagePrediction['p3_conf'] > 1])

Empty DataFrame
Columns: [tweet_id, jpg_url, img_num, p1, p1_conf, p1_dog, p2, p2_conf, p2_dog, p3, p3_conf, p3_dog]
Index: []
Empty DataFrame
Columns: [tweet_id, jpg_url, img_num, p1, p1_conf, p1_dog, p2, p2_conf, p2_dog, p3, p3_conf, p3_dog]
Index: []


## Tidleness issue

In [31]:
#1: Combine dog type columns
# -After read some twitters, we found that dog types are from twitter text's key word. 
# So, we use find function to analyse string, if there is a keyword in such string, we 
# append the keyword to list, and use the new created list to create a row called rating.
stages=[]
for each in df_WeRateDogs['text']:
    if each.lower().find('doggo') != -1 and each.lower().find('floofer') == -1 and each.lower().find('pupper') == -1 and each.lower().find('puppo') == -1:
        stages.append('doggo')
    elif each.lower().find('floofer') != -1 and each.lower().find('doggo') == -1 and each.lower().find('pupper') == -1 and each.lower().find('puppo') == -1:
        stages.append('floofer')
    elif each.lower().find('pupper') != -1 and each.lower().find('doggo') == -1 and each.lower().find('floofer') == -1 and each.lower().find('puppo') == -1:
        stages.append('pupper')
    elif each.lower().find('puppo') != -1 and each.lower().find('doggo') == -1 and each.lower().find('floofer') == -1 and each.lower().find('pupper') == -1:
        stages.append('puppo')
    elif each.lower().find('puppo') == -1 and each.lower().find('doggo') == -1 and each.lower().find('floofer') == -1 and each.lower().find('pupper') == -1:
        stages.append(np.nan)
    else:
        stages.append('critical')
df_WeRateDogs['stages'] = stages

In [32]:
df_WeRateDogs[df_WeRateDogs['stages'] == 'critical'][['tweet_id','text']]

,tweet_id,text
172,858843525470990336,I have stumbled puppon a doggo painting party....
191,855851453814013952,Here's a puppo participating in the #ScienceMa...
200,854010172552949760,"At first I thought this was a shy doggo, but i..."
460,817777686764523521,"This is Dido. She's playing the lead role in ""..."
531,808106460588765185,Here we have Burke (pupper) and Dexter (doggo)...
565,802265048156610565,"Like doggo, like pupper version 2. Both 11/10 ..."
575,801115127852503040,This is Bones. He's being haunted by another d...
705,785639753186217984,This is Pinot. He's a sophisticated doggo. You...
733,781308096455073793,"Pupper butt 1, Doggo 0. Both 12/10 https://t.c..."
889,759793422261743616,"Meet Maggie &amp; Lila. Maggie is the doggo, L..."


In [33]:
index = [172,191,200,460,531,565,575,705,733,889,956,1063,1113]
for each in df_WeRateDogs[df_WeRateDogs['stages'] == 'critical']['text']:
    print(each)

I have stumbled puppon a doggo painting party. They're looking to be the next Pupcasso or Puppollock. All 13/10 would put it on the fridge https://t.co/cUeDMlHJbq
Here's a puppo participating in the #ScienceMarch. Cleverly disguising her own doggo agenda. 13/10 would keep the planet habitable for https://t.co/cMhq16isel
At first I thought this was a shy doggo, but it's actually a Rare Canadian Floofer Owl. Amateurs would confuse the two. 11/10 only send dogs https://t.co/TXdT3tmuYk
This is Dido. She's playing the lead role in "Pupper Stops to Catch Snow Before Resuming Shadow Box with Dried Apple." 13/10 (IG: didodoggo) https://t.co/m7isZrOBX7
Here we have Burke (pupper) and Dexter (doggo). Pupper wants to be exactly like doggo. Both 12/10 would pet at same time https://t.co/ANBpEYHaho
Like doggo, like pupper version 2. Both 11/10 https://t.co/9IxWAXFqze
This is Bones. He's being haunted by another doggo of roughly the same size. 12/10 deep breaths pupper everything's fine https://t.co

In [34]:
types = ['doggo','puppo','floofer','pupper','multiple','pupper',
         'doggo','doggo','pupper','critical','multiple',np.nan,'multiple','multiple']


dicts = dict(zip(index,types))

for k,v in dicts.items():
    df_WeRateDogs.at[k,'stages'] = v

In [35]:
name=[]
for each in df_WeRateDogs['name']:
    if each.islower() or each == 'None':
        name.append(np.nan)
    else:
        name.append(each)
df_WeRateDogs['name'] = name

In [36]:
#2: Combine numerator and denominator as rating
# -After deal with problems in denominator and numerator, use denominator divide numerator.
df_WeRateDogs['rating'] = df_WeRateDogs['rating_numerator']/df_WeRateDogs['rating_denominator']
df_WeRateDogs.drop(['rating_numerator','rating_denominator'], axis=1, inplace=True)

In [37]:
df_WeRateDogs = df_WeRateDogs[['tweet_id','timestamp','text','name','stages','rating']]
#test
df_WeRateDogs

,tweet_id,timestamp,text,name,stages,rating
0,892420643555336193,2017-08-01 16:23:56,This is Phineas. He's a mystical boy. Only eve...,Phineas,NaN,1.3
1,892177421306343426,2017-08-01 00:17:27,This is Tilly. She's just checking pup on you....,Tilly,NaN,1.3
2,891815181378084864,2017-07-31 00:18:03,This is Archie. He is a rare Norwegian Pouncin...,Archie,NaN,1.2
3,891689557279858688,2017-07-30 15:58:51,This is Darla. She commenced a snooze mid meal...,Darla,NaN,1.3
4,891327558926688256,2017-07-29 16:00:24,This is Franklin. He would like you to stop ca...,Franklin,NaN,1.2
5,891087950875897856,2017-07-29 00:08:17,Here we have a majestic great white breaching ...,NaN,NaN,1.3
6,890971913173991426,2017-07-28 16:27:12,Meet Jax. He enjoys ice cream so much he gets ...,Jax,NaN,1.3
7,890729181411237888,2017-07-28 00:22:40,When you watch your owner call another dog a g...,NaN,NaN,1.3
8,890609185150312448,2017-07-27 16:25:51,This is Zoey. She doesn't want to be one of th...,Zoey,NaN,1.3
9,890240255349198849,2017-07-26 15:59:51,This is Cassie. She is a college pup. Studying...,Cassie,doggo,1.4


In [38]:
#Combine 3 datasets to a whole big master dataset
df_WeRateDogs.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2117 entries, 0 to 2355
Data columns (total 6 columns):
tweet_id     2117 non-null object
timestamp    2117 non-null datetime64[ns]
text         2117 non-null object
name         1391 non-null object
stages       376 non-null object
rating       2117 non-null object
dtypes: datetime64[ns](1), object(5)
memory usage: 195.8+ KB


In [39]:
df_imagePrediction.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2009 entries, 0 to 2074
Data columns (total 12 columns):
tweet_id    2009 non-null object
jpg_url     2009 non-null object
img_num     2009 non-null int64
p1          2009 non-null object
p1_conf     2009 non-null float64
p1_dog      2009 non-null bool
p2          2009 non-null object
p2_conf     2009 non-null float64
p2_dog      2009 non-null bool
p3          2009 non-null object
p3_conf     2009 non-null float64
p3_dog      2009 non-null bool
dtypes: bool(3), float64(3), int64(1), object(5)
memory usage: 162.8+ KB


In [52]:
df_tweepy.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2354 entries, 0 to 2353
Data columns (total 3 columns):
tweet_id           2354 non-null object
retweet_count      2354 non-null int64
favourite_count    2354 non-null int64
dtypes: int64(2), object(1)
memory usage: 55.2+ KB


In [78]:
temp = pd.merge(df_WeRateDogs,df_imagePrediction,on='tweet_id')
temp2 = pd.merge(df_tweepy,temp,on=['tweet_id'])

In [79]:
temp2

,tweet_id,retweet_count,favourite_count,timestamp,text,name,stages,rating,jpg_url,img_num,p1,p1_conf,p1_dog,p2,p2_conf,p2_dog,p3,p3_conf,p3_dog


In [80]:
temp2 = pd.merge(df_tweepy,temp,on=['tweet_id'],how='left')

In [81]:
temp2.head(3)

,tweet_id,retweet_count,favourite_count,timestamp,text,name,stages,rating,jpg_url,img_num,p1,p1_conf,p1_dog,p2,p2_conf,p2_dog,p3,p3_conf,p3_dog
0,892420643555336193,8853,39467,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,892177421306343426,6514,33819,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,891815181378084864,4328,25461,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [82]:
temp2 = pd.merge(df_tweepy,temp,on=['tweet_id'],how='right')

In [83]:
temp2.head(3)

,tweet_id,retweet_count,favourite_count,timestamp,text,name,stages,rating,jpg_url,img_num,p1,p1_conf,p1_dog,p2,p2_conf,p2_dog,p3,p3_conf,p3_dog
0,892420643555336193,NaN,NaN,2017-08-01 16:23:56,This is Phineas. He's a mystical boy. Only eve...,Phineas,NaN,1.3,https://pbs.twimg.com/media/DGKD1-bXoAAIAUK.jpg,1,orange,0.097049,False,bagel,0.008585,False,banana,0.007611,False
1,892177421306343426,NaN,NaN,2017-08-01 00:17:27,This is Tilly. She's just checking pup on you....,Tilly,NaN,1.3,https://pbs.twimg.com/media/DGGmoV4XsAAUL6n.jpg,1,Chihuahua,0.323581,True,Pekinese,0.009065,True,papillon,0.006896,True
2,891815181378084864,NaN,NaN,2017-07-31 00:18:03,This is Archie. He is a rare Norwegian Pouncin...,Archie,NaN,1.2,https://pbs.twimg.com/media/DGBdLU1WsAANxJ9.jpg,1,Chihuahua,0.716012,True,malamute,0.007825,True,kelpie,0.003138,True


# Visualisation

# Act Report